In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np 
import pandas as pd 
import pickle
import datetime
import xgboost as xgb
from IPython.display import display

In [3]:
# dec_orders = pd.read_csv('orders_before_dec.csv', sep='|')
jan_orders = pd.read_csv('orders_before_jan.csv', sep='|')

# submission_dec = pd.read_csv('submission_dec.csv', sep='|')
submission_jan = pd.read_csv('submission_jan.csv', sep='|')

In [4]:
# Remember to try different train test splitts!!!
# Label all data or just a subset?
# Look at meta cost?

In [5]:
#1
# jan_orders = pd.merge(jan_orders, submission_jan, on=['itemID', 'userID'], how='left')
# display(jan_orders)

In [6]:
# OK å gjøre dette før test splitt, hva kan være med i test data?

# 2
jan_orders = pd.merge(jan_orders, submission_jan, on=['itemID', 'userID'], how='inner')
display(jan_orders)
# jan_orders.to_csv('understand_data.csv', sep='|', index=False)

,date,userID,itemID,order,prediction
0,2020-06-01,31089,23582,1,NaN
1,2020-08-10,31089,23582,1,NaN
2,2020-10-17,31089,23582,1,NaN
3,2020-11-30,31089,23582,2,NaN
4,2020-06-01,2635,31850,3,NaN
...,...,...,...,...,...
27730,2020-12-29,29515,26027,1,NaN
27731,2020-12-29,15207,8265,3,NaN
27732,2020-12-30,43101,29426,1,NaN
27733,2020-12-30,31513,8366,1,NaN


In [7]:
jan_orders = jan_orders.sort_values(by=['date'], ascending=True)
display(jan_orders)

,date,userID,itemID,order,prediction
0,2020-06-01,31089,23582,1,NaN
206,2020-06-01,16461,12663,1,NaN
530,2020-06-01,3745,19382,1,NaN
209,2020-06-01,16461,2169,1,NaN
527,2020-06-01,6494,26830,1,NaN
...,...,...,...,...,...
3237,2020-12-31,42163,19746,2,NaN
3243,2020-12-31,42163,24763,3,NaN
19049,2020-12-31,34669,18863,1,NaN
19113,2020-12-31,40427,12958,1,NaN


In [8]:
jan_orders = jan_orders[jan_orders.duplicated(subset=['userID','itemID'], keep=False)]
display(jan_orders)

,date,userID,itemID,order,prediction
0,2020-06-01,31089,23582,1,NaN
206,2020-06-01,16461,12663,1,NaN
530,2020-06-01,3745,19382,1,NaN
209,2020-06-01,16461,2169,1,NaN
527,2020-06-01,6494,26830,1,NaN
...,...,...,...,...,...
26983,2020-12-31,34014,1284,1,NaN
3237,2020-12-31,42163,19746,2,NaN
3243,2020-12-31,42163,24763,3,NaN
19049,2020-12-31,34669,18863,1,NaN


In [9]:
jan_orders.drop(['prediction'], axis = 1, inplace = True)

In [10]:
# Join with items.csv
items = pd.read_csv('items.csv', sep='|')

jan_orders = pd.merge(jan_orders, items, on=['itemID'], how='left')
jan_orders.drop(['categories'], axis = 1, inplace = True)
display(jan_orders)

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5
0,2020-06-01,31089,23582,1,569,4,0,21,3,52
1,2020-06-01,16461,12663,1,363,10,0,503,3,17
2,2020-06-01,3745,19382,1,290,10,1,503,0,122
3,2020-06-01,16461,2169,1,1301,4,0,148,0,144
4,2020-06-01,6494,26830,1,391,10,1,491,0,156
...,...,...,...,...,...,...,...,...,...,...
27070,2020-12-31,34014,1284,1,194,10,0,503,0,17
27071,2020-12-31,42163,19746,2,485,10,0,421,3,3
27072,2020-12-31,42163,24763,3,186,6,0,207,0,17
27073,2020-12-31,34669,18863,1,393,4,0,485,3,44


In [11]:
# Regn ut average tid mellom kjøp og så label basert på dette.  og så predict git tid fra siste kjøp, slik som en av gruppene gjorde. 
# X_train are all orders from 1. of June to 15. of November. 24 weeks

# X_train = jan_orders[(jan_orders['date'] <= '2020-11-15')]
# X_train = X_train.sort_values(by=['userID', 'itemID'], ascending=True)
# display(X_train)

# # X_test are all orders from 16. of November to 27. of December. 6 weeks
# X_test = jan_orders[(jan_orders['date'] >= '2020-11-16') & (jan_orders['date'] <= '2020-12-27')]
# X_test = X_test.sort_values(by=['userID', 'itemID'], ascending=True)
# display(X_test)

In [12]:
# ikke sikker på om det er lov, men prøver å lable basert på all data. 
X_train = jan_orders[(jan_orders['date'] < '2020-12-31')]
X_train = X_train.sort_values(by=['userID', 'itemID'], ascending=True)
display(X_train)

# X_test are all orders from 16. of November to 27. of December. 6 weeks
# X_test = jan_orders[(jan_orders['date'] >= '2020-11-16') & (jan_orders['date'] <= '2020-12-27')]
# X_test = X_test.sort_values(by=['userID', 'itemID'], ascending=True)
# display(X_test)

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5
593,2020-06-05,0,20664,1,408,4,0,284,0,66
16052,2020-10-09,0,20664,1,408,4,0,284,0,66
24588,2020-12-11,0,20664,1,408,4,0,284,0,66
21329,2020-11-20,0,28231,1,193,4,3,468,3,108
24599,2020-12-11,0,28231,1,193,4,3,468,3,108
...,...,...,...,...,...,...,...,...,...,...
22735,2020-11-29,46127,7963,2,1111,4,0,485,3,154
25102,2020-12-14,46127,7963,1,1111,4,0,485,3,154
430,2020-06-03,46130,395,1,745,4,0,399,3,163
19969,2020-11-09,46130,395,2,745,4,0,399,3,163


In [13]:
# Behold duplikater
X_train = X_train[X_train.duplicated(subset=['userID','itemID'], keep=False)]
display(X_train)

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5
593,2020-06-05,0,20664,1,408,4,0,284,0,66
16052,2020-10-09,0,20664,1,408,4,0,284,0,66
24588,2020-12-11,0,20664,1,408,4,0,284,0,66
21329,2020-11-20,0,28231,1,193,4,3,468,3,108
24599,2020-12-11,0,28231,1,193,4,3,468,3,108
...,...,...,...,...,...,...,...,...,...,...
22735,2020-11-29,46127,7963,2,1111,4,0,485,3,154
25102,2020-12-14,46127,7963,1,1111,4,0,485,3,154
430,2020-06-03,46130,395,1,745,4,0,399,3,163
19969,2020-11-09,46130,395,2,745,4,0,399,3,163


In [14]:
X_train['date']=pd.to_datetime(X_train['date'])
tmp3 = X_train.copy(deep = True)

In [15]:
# tmp = X_train.groupby(['userID', 'itemID'])['date'].apply(lambda x: (x.diff().dt.days).mean())
tmp = tmp3.groupby(['userID', 'itemID'])['date'].apply(lambda x: (x.diff().dt.days).mean()).reset_index(name='days_diff')

In [16]:
# display(X_train)
tmp['days_diff'] = 2*tmp['days_diff']
# tmp = tmp.sort_values(by=['days_diff'], ascending=True) # Some orders are from the same day and not repeated after that!!!
display(tmp)
# tmp.drop(tmp[tmp['days_diff'] == 0].index, inplace=True)
# display(tmp)

,userID,itemID,days_diff
0,0,20664,189.0
1,0,28231,42.0
2,13,2690,134.0
3,15,1299,108.0
4,15,20968,79.0
...,...,...,...
9265,46118,20106,84.5
9266,46124,19677,69.5
9267,46125,12878,328.0
9268,46127,7963,80.0


In [17]:
tmp3.drop(['date', 'brand', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5'], axis = 1, inplace = True )
display(tmp3)
tmp3['order_amount'] = tmp3.groupby(['userID','itemID'])['order'].transform('sum')
tmp3.drop(['order'], axis = 1, inplace = True)
display(tmp3)
tmp7 = tmp3.copy(deep=True)
tmp7 = tmp7.groupby(['userID', 'itemID']).size().reset_index(name='counts')
# tmp3 = tmp3.sort_values(by=['counts'], ascending=True)
# tmp2 = print(X_train.groupby(['userID', 'itemID']).count())
# display(tmp3)
# display(tmp7)
tmp3 = pd.merge(tmp3, tmp7, on=['itemID', 'userID'], how='left')
display(tmp3)

,userID,itemID,order
593,0,20664,1
16052,0,20664,1
24588,0,20664,1
21329,0,28231,1
24599,0,28231,1
...,...,...,...
22735,46127,7963,2
25102,46127,7963,1
430,46130,395,1
19969,46130,395,2


,userID,itemID,order_amount
593,0,20664,3
16052,0,20664,3
24588,0,20664,3
21329,0,28231,2
24599,0,28231,2
...,...,...,...
22735,46127,7963,8
25102,46127,7963,8
430,46130,395,5
19969,46130,395,5


,userID,itemID,order_amount,counts
0,0,20664,3,3
1,0,20664,3,3
2,0,20664,3,3
3,0,28231,2,2
4,0,28231,2,2
...,...,...,...,...
27007,46127,7963,8,5
27008,46127,7963,8,5
27009,46130,395,5,3
27010,46130,395,5,3


In [18]:
# Merge de to. Del days_dif på counts. Har da average. 
# hvis average er 7 dager eller mindre set lik 1 hvis, over 7 under 15 set lik 2 osv. fra siste dag, opp mot januar månde
# Må gjøre det samme for december for å se at det generalisrere greit

In [19]:
tmp2 = pd.merge(tmp3, tmp, on=['itemID', 'userID'], how='right') 
tmp2

,userID,itemID,order_amount,counts,days_diff
0,0,20664,3,3,189.0
1,0,20664,3,3,189.0
2,0,20664,3,3,189.0
3,0,28231,2,2,42.0
4,0,28231,2,2,42.0
...,...,...,...,...,...
27007,46127,7963,8,5,80.0
27008,46127,7963,8,5,80.0
27009,46130,395,5,3,200.0
27010,46130,395,5,3,200.0


In [20]:
tmp2['avg_purchase_time'] = tmp2.days_diff.div(tmp2.counts)
tmp2['avg_order_amount'] = tmp2.order_amount.div(tmp2.counts)
tmp2

,userID,itemID,order_amount,counts,days_diff,avg_purchase_time,avg_order_amount
0,0,20664,3,3,189.0,63.000000,1.000000
1,0,20664,3,3,189.0,63.000000,1.000000
2,0,20664,3,3,189.0,63.000000,1.000000
3,0,28231,2,2,42.0,21.000000,1.000000
4,0,28231,2,2,42.0,21.000000,1.000000
...,...,...,...,...,...,...,...
27007,46127,7963,8,5,80.0,16.000000,1.600000
27008,46127,7963,8,5,80.0,16.000000,1.600000
27009,46130,395,5,3,200.0,66.666667,1.666667
27010,46130,395,5,3,200.0,66.666667,1.666667


In [21]:
display(X_train)
X_train = pd.merge(X_train, tmp2, on=['itemID', 'userID'], how='left') 
X_train.drop(X_train[X_train['days_diff'] == 0].index, inplace=True)
X_train.drop(['counts', 'order_amount','days_diff'], axis = 1, inplace = True)
X_train

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5
593,2020-06-05,0,20664,1,408,4,0,284,0,66
16052,2020-10-09,0,20664,1,408,4,0,284,0,66
24588,2020-12-11,0,20664,1,408,4,0,284,0,66
21329,2020-11-20,0,28231,1,193,4,3,468,3,108
24599,2020-12-11,0,28231,1,193,4,3,468,3,108
...,...,...,...,...,...,...,...,...,...,...
22735,2020-11-29,46127,7963,2,1111,4,0,485,3,154
25102,2020-12-14,46127,7963,1,1111,4,0,485,3,154
430,2020-06-03,46130,395,1,745,4,0,399,3,163
19969,2020-11-09,46130,395,2,745,4,0,399,3,163


,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,avg_purchase_time,avg_order_amount
0,2020-06-05,0,20664,1,408,4,0,284,0,66,63.000000,1.000000
1,2020-06-05,0,20664,1,408,4,0,284,0,66,63.000000,1.000000
2,2020-06-05,0,20664,1,408,4,0,284,0,66,63.000000,1.000000
3,2020-10-09,0,20664,1,408,4,0,284,0,66,63.000000,1.000000
4,2020-10-09,0,20664,1,408,4,0,284,0,66,63.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
94619,2020-11-09,46130,395,2,745,4,0,399,3,163,66.666667,1.666667
94620,2020-11-09,46130,395,2,745,4,0,399,3,163,66.666667,1.666667
94621,2020-12-20,46130,395,2,745,4,0,399,3,163,66.666667,1.666667
94622,2020-12-20,46130,395,2,745,4,0,399,3,163,66.666667,1.666667


In [22]:
# We have date up until 15 of november. Want column with last order date for each userID, itemID pair

In [23]:
tmp4 = X_train.copy(deep=True)

In [24]:
tmp4 = tmp4.drop_duplicates(['userID', 'itemID'],keep='last')
tmp4.drop([ 'brand', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5'], axis = 1, inplace=True)
tmp4.rename(columns = {'date':'last_date'}, inplace = True)
tmp4['y_label'] = np.nan
tmp4

,last_date,userID,itemID,order,avg_purchase_time,avg_order_amount,y_label
8,2020-12-11,0,20664,1,63.000000,1.000000,NaN
12,2020-12-11,0,28231,1,21.000000,1.000000,NaN
28,2020-12-24,13,2690,1,33.500000,1.000000,NaN
37,2020-12-26,15,1299,1,36.000000,1.000000,NaN
46,2020-12-26,15,20968,1,26.333333,1.000000,NaN
...,...,...,...,...,...,...,...
94560,2020-12-23,46118,20106,1,16.900000,1.000000,NaN
94585,2020-12-25,46124,19677,1,13.900000,1.000000,NaN
94589,2020-11-15,46125,12878,1,164.000000,1.000000,NaN
94614,2020-12-14,46127,7963,1,16.000000,1.600000,NaN


In [25]:
tmp4['avg_purchase_time_per_order'] = tmp4.avg_purchase_time.div(tmp4.avg_order_amount)
display(tmp4)

,last_date,userID,itemID,order,avg_purchase_time,avg_order_amount,y_label,avg_purchase_time_per_order
8,2020-12-11,0,20664,1,63.000000,1.000000,NaN,63.000000
12,2020-12-11,0,28231,1,21.000000,1.000000,NaN,21.000000
28,2020-12-24,13,2690,1,33.500000,1.000000,NaN,33.500000
37,2020-12-26,15,1299,1,36.000000,1.000000,NaN,36.000000
46,2020-12-26,15,20968,1,26.333333,1.000000,NaN,26.333333
...,...,...,...,...,...,...,...,...
94560,2020-12-23,46118,20106,1,16.900000,1.000000,NaN,16.900000
94585,2020-12-25,46124,19677,1,13.900000,1.000000,NaN,13.900000
94589,2020-11-15,46125,12878,1,164.000000,1.000000,NaN,164.000000
94614,2020-12-14,46127,7963,1,16.000000,1.600000,NaN,10.000000


In [26]:
tmp4['avg_purchase_time'] = tmp4['avg_purchase_time_per_order'] * tmp4['order']
tmp4

,last_date,userID,itemID,order,avg_purchase_time,avg_order_amount,y_label,avg_purchase_time_per_order
8,2020-12-11,0,20664,1,63.000000,1.000000,NaN,63.000000
12,2020-12-11,0,28231,1,21.000000,1.000000,NaN,21.000000
28,2020-12-24,13,2690,1,33.500000,1.000000,NaN,33.500000
37,2020-12-26,15,1299,1,36.000000,1.000000,NaN,36.000000
46,2020-12-26,15,20968,1,26.333333,1.000000,NaN,26.333333
...,...,...,...,...,...,...,...,...
94560,2020-12-23,46118,20106,1,16.900000,1.000000,NaN,16.900000
94585,2020-12-25,46124,19677,1,13.900000,1.000000,NaN,13.900000
94589,2020-11-15,46125,12878,1,164.000000,1.000000,NaN,164.000000
94614,2020-12-14,46127,7963,1,10.000000,1.600000,NaN,10.000000


In [27]:
# Rund opp avg_purchase_time til heltal

In [28]:
# tmp5.head(40)
# tmp5 = tmp5.sort_values(by=['last_date', 'avg_purchase_time'], ascending=True)
# Next purchase date. Loop until in january for . If not then predict 0
# tmp5.to_csv('inspect_last_date_avg_purchase.csv', sep='|', index=False)
tmp4['ceil_time'] = tmp4['avg_purchase_time'].apply(np.ceil)
tmp4

,last_date,userID,itemID,order,avg_purchase_time,avg_order_amount,y_label,avg_purchase_time_per_order,ceil_time
8,2020-12-11,0,20664,1,63.000000,1.000000,NaN,63.000000,63.0
12,2020-12-11,0,28231,1,21.000000,1.000000,NaN,21.000000,21.0
28,2020-12-24,13,2690,1,33.500000,1.000000,NaN,33.500000,34.0
37,2020-12-26,15,1299,1,36.000000,1.000000,NaN,36.000000,36.0
46,2020-12-26,15,20968,1,26.333333,1.000000,NaN,26.333333,27.0
...,...,...,...,...,...,...,...,...,...
94560,2020-12-23,46118,20106,1,16.900000,1.000000,NaN,16.900000,17.0
94585,2020-12-25,46124,19677,1,13.900000,1.000000,NaN,13.900000,14.0
94589,2020-11-15,46125,12878,1,164.000000,1.000000,NaN,164.000000,164.0
94614,2020-12-14,46127,7963,1,10.000000,1.600000,NaN,10.000000,10.0


In [29]:
# Last day + average tar deg til januar???
# Lag colone, filtrer på det som er i januar og label slik som tidligere!

tmp4['pred_date'] = (tmp4['last_date'] + pd.to_timedelta(tmp4['avg_purchase_time'], unit='D')).dt.date # Ikke ceil_time?
tmp4

,last_date,userID,itemID,order,avg_purchase_time,avg_order_amount,y_label,avg_purchase_time_per_order,ceil_time,pred_date
8,2020-12-11,0,20664,1,63.000000,1.000000,NaN,63.000000,63.0,2021-02-12
12,2020-12-11,0,28231,1,21.000000,1.000000,NaN,21.000000,21.0,2021-01-01
28,2020-12-24,13,2690,1,33.500000,1.000000,NaN,33.500000,34.0,2021-01-26
37,2020-12-26,15,1299,1,36.000000,1.000000,NaN,36.000000,36.0,2021-01-31
46,2020-12-26,15,20968,1,26.333333,1.000000,NaN,26.333333,27.0,2021-01-21
...,...,...,...,...,...,...,...,...,...,...
94560,2020-12-23,46118,20106,1,16.900000,1.000000,NaN,16.900000,17.0,2021-01-08
94585,2020-12-25,46124,19677,1,13.900000,1.000000,NaN,13.900000,14.0,2021-01-07
94589,2020-11-15,46125,12878,1,164.000000,1.000000,NaN,164.000000,164.0,2021-04-28
94614,2020-12-14,46127,7963,1,10.000000,1.600000,NaN,10.000000,10.0,2020-12-24


In [30]:
tmp4['pred_date'] = pd.to_datetime(tmp4['pred_date'])

In [31]:
tmp4.drop(['last_date', 'avg_purchase_time', 'ceil_time', 'order', 'avg_order_amount', 'avg_purchase_time_per_order'], axis = 1, inplace=True)
tmp4

,userID,itemID,y_label,pred_date
8,0,20664,NaN,2021-02-12
12,0,28231,NaN,2021-01-01
28,13,2690,NaN,2021-01-26
37,15,1299,NaN,2021-01-31
46,15,20968,NaN,2021-01-21
...,...,...,...,...
94560,46118,20106,NaN,2021-01-08
94585,46124,19677,NaN,2021-01-07
94589,46125,12878,NaN,2021-04-28
94614,46127,7963,NaN,2020-12-24


In [32]:

tmp4['y_label'] = np.where((tmp4['pred_date'] > '2021-01-28'),0 ,tmp4.y_label)
tmp4['y_label'] = np.where((tmp4['pred_date'] < '2021-01-01'),0 ,tmp4.y_label)
tmp4['y_label'] = np.where((tmp4['pred_date'] >= '2021-01-01') & (tmp4['pred_date'] <= '2021-01-07'),1 ,tmp4.y_label)
tmp4['y_label'] = np.where((tmp4['pred_date'] >= '2021-01-08') & (tmp4['pred_date'] <= '2021-01-14'),2 ,tmp4.y_label)
tmp4['y_label'] = np.where((tmp4['pred_date'] >= '2021-01-15') & (tmp4['pred_date'] <= '2021-01-21'),3 ,tmp4.y_label)
tmp4['y_label'] = np.where((tmp4['pred_date'] >= '2021-01-22') & (tmp4['pred_date'] <= '2021-01-28'),4 ,tmp4.y_label)


#tror jeg alt har droppet duplikater, men må endre på dette her hvis vi har de med igjen
tmp4.drop(['pred_date'], axis = 1, inplace=True)
tmp4.rename(columns = {'y_label':'prediction'}, inplace = True)
tmp4

,userID,itemID,prediction
8,0,20664,0.0
12,0,28231,1.0
28,13,2690,4.0
37,15,1299,0.0
46,15,20968,3.0
...,...,...,...
94560,46118,20106,2.0
94585,46124,19677,1.0
94589,46125,12878,0.0
94614,46127,7963,0.0


In [33]:
tmp5 = tmp4.prediction.value_counts()
tmp5

0.0    6758
1.0     748
2.0     644
3.0     607
4.0     507
Name: prediction, dtype: int64

# The change that was made gives worse results 

##### Test to see if labeling is a good prediction, if not don't use the block below

In [34]:
# # Testing labeling
# # Make/addapt to submission file. 
# submission_jan.drop(['prediction'], axis = 1, inplace = True) 
# submission_jan = pd.merge(tmp4, submission_jan, on=['userID', 'itemID'], how='right')

# #Fill inn 0 for the (user, items) combinations we have not seen before (Not seen in january. Might have to small test set)
# submission_jan['prediction'] = submission_jan['prediction'].fillna(0) 
# submission_jan.drop_duplicates(subset=['userID', 'itemID'], keep='first', inplace = True)
# # submission_jan.drop(['order', 'date', 'brand', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5'], axis = 1, inplace = True)
# display(submission_jan)

# tmp5 = submission_jan.prediction.value_counts()
# tmp5

In [35]:
Data = pd.merge(X_train, tmp4, on=['itemID', 'userID'], how='left')
Data.drop(['avg_purchase_time'], axis = 1, inplace=True)
display(Data)

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,avg_order_amount,prediction
0,2020-06-05,0,20664,1,408,4,0,284,0,66,1.000000,0.0
1,2020-06-05,0,20664,1,408,4,0,284,0,66,1.000000,0.0
2,2020-06-05,0,20664,1,408,4,0,284,0,66,1.000000,0.0
3,2020-10-09,0,20664,1,408,4,0,284,0,66,1.000000,0.0
4,2020-10-09,0,20664,1,408,4,0,284,0,66,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
94595,2020-11-09,46130,395,2,745,4,0,399,3,163,1.666667,0.0
94596,2020-11-09,46130,395,2,745,4,0,399,3,163,1.666667,0.0
94597,2020-12-20,46130,395,2,745,4,0,399,3,163,1.666667,0.0
94598,2020-12-20,46130,395,2,745,4,0,399,3,163,1.666667,0.0


In [36]:
# Train/test splitt:
X_train = Data[(Data['date'] <= '2020-11-15')]
X_train = X_train.sort_values(by=['userID', 'itemID'], ascending=True)
display(X_train)

# X_test are all orders from 16. of November to 27. of December. 6 weeks
X_test = Data[(Data['date'] >= '2020-11-16') & (Data['date'] <= '2020-12-27')]
X_test = X_test.sort_values(by=['userID', 'itemID'], ascending=True)

tmp7 = X_test.prediction.value_counts()  #se på dette!
display(tmp7)
X_test.drop(['prediction'], axis = 1, inplace=True) #teste om modelen lører på denne fasiten hvis vi vil??? (altså sette denne som fasit)
display(X_test)

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,avg_order_amount,prediction
0,2020-06-05,0,20664,1,408,4,0,284,0,66,1.000000,0.0
1,2020-06-05,0,20664,1,408,4,0,284,0,66,1.000000,0.0
2,2020-06-05,0,20664,1,408,4,0,284,0,66,1.000000,0.0
3,2020-10-09,0,20664,1,408,4,0,284,0,66,1.000000,0.0
4,2020-10-09,0,20664,1,408,4,0,284,0,66,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
94592,2020-06-03,46130,395,1,745,4,0,399,3,163,1.666667,0.0
94593,2020-06-03,46130,395,1,745,4,0,399,3,163,1.666667,0.0
94594,2020-11-09,46130,395,2,745,4,0,399,3,163,1.666667,0.0
94595,2020-11-09,46130,395,2,745,4,0,399,3,163,1.666667,0.0


0.0    12198
1.0     2740
2.0     1860
3.0     1557
4.0     1186
Name: prediction, dtype: int64

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,avg_order_amount
6,2020-12-11,0,20664,1,408,4,0,284,0,66,1.000000
7,2020-12-11,0,20664,1,408,4,0,284,0,66,1.000000
8,2020-12-11,0,20664,1,408,4,0,284,0,66,1.000000
9,2020-11-20,0,28231,1,193,4,3,468,3,108,1.000000
10,2020-11-20,0,28231,1,193,4,3,468,3,108,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
94589,2020-12-14,46127,7963,1,1111,4,0,485,3,154,1.600000
94590,2020-12-14,46127,7963,1,1111,4,0,485,3,154,1.600000
94597,2020-12-20,46130,395,2,745,4,0,399,3,163,1.666667
94598,2020-12-20,46130,395,2,745,4,0,399,3,163,1.666667


In [37]:
def convert_dates(x):
    x['date']=pd.to_datetime(x['date'])
    x['month']=x['date'].dt.month
    x['day'] = x['date'].dt.day
    x['dayofweek']=x['date'].dt.dayofweek
    x.pop('date')
    return x

X_train = convert_dates(X_train)
X_test = convert_dates(X_test)
display(X_train)

,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,avg_order_amount,prediction,month,day,dayofweek
0,0,20664,1,408,4,0,284,0,66,1.000000,0.0,6,5,4
1,0,20664,1,408,4,0,284,0,66,1.000000,0.0,6,5,4
2,0,20664,1,408,4,0,284,0,66,1.000000,0.0,6,5,4
3,0,20664,1,408,4,0,284,0,66,1.000000,0.0,10,9,4
4,0,20664,1,408,4,0,284,0,66,1.000000,0.0,10,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94592,46130,395,1,745,4,0,399,3,163,1.666667,0.0,6,3,2
94593,46130,395,1,745,4,0,399,3,163,1.666667,0.0,6,3,2
94594,46130,395,2,745,4,0,399,3,163,1.666667,0.0,11,9,0
94595,46130,395,2,745,4,0,399,3,163,1.666667,0.0,11,9,0


In [38]:
# Lag X_train og y_train. putt de inn i en random forest balansert etter vekter og se om du får noe. 
# hvis ikke bruk alle eksemplene til trening.
tmp6 = X_train.copy(deep = True)
y_train = tmp6[['prediction']]
display(y_train)

tmp6.drop(['prediction'], axis = 1, inplace = True)
X_train = tmp6
display(X_train)

,prediction
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
94592,0.0
94593,0.0
94594,0.0
94595,0.0


,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,avg_order_amount,month,day,dayofweek
0,0,20664,1,408,4,0,284,0,66,1.000000,6,5,4
1,0,20664,1,408,4,0,284,0,66,1.000000,6,5,4
2,0,20664,1,408,4,0,284,0,66,1.000000,6,5,4
3,0,20664,1,408,4,0,284,0,66,1.000000,10,9,4
4,0,20664,1,408,4,0,284,0,66,1.000000,10,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94592,46130,395,1,745,4,0,399,3,163,1.666667,6,3,2
94593,46130,395,1,745,4,0,399,3,163,1.666667,6,3,2
94594,46130,395,2,745,4,0,399,3,163,1.666667,11,9,0
94595,46130,395,2,745,4,0,399,3,163,1.666667,11,9,0


##### Testing different algorithms

In [39]:
X_training = X_train.copy(deep=True)
y_training = y_train.copy(deep=True)

In [40]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X_train, y_train)
clf = RandomForestClassifier(random_state=42, class_weight={0: 1, 1: 100, 2: 100, 3: 100, 4: 100})
clf.fit(X_training, y_training) 

RandomForestClassifier(class_weight={0: 1, 1: 100, 2: 100, 3: 100, 4: 100},
                       random_state=42)

In [41]:
# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier(class_weight={0: 1, 1: 100, 2: 100, 3: 100, 4: 100})
# clf = clf.fit(X_training,y_training)

In [42]:
pred = clf.predict(X_test)

In [43]:
X_test['prediction'] = pred
display(X_test)

tmp4 = X_test.prediction.value_counts()  #se på dette!
tmp4

,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,avg_order_amount,month,day,dayofweek,prediction
6,0,20664,1,408,4,0,284,0,66,1.000000,12,11,4,0.0
7,0,20664,1,408,4,0,284,0,66,1.000000,12,11,4,0.0
8,0,20664,1,408,4,0,284,0,66,1.000000,12,11,4,0.0
9,0,28231,1,193,4,3,468,3,108,1.000000,11,20,4,0.0
10,0,28231,1,193,4,3,468,3,108,1.000000,11,20,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94589,46127,7963,1,1111,4,0,485,3,154,1.600000,12,14,0,0.0
94590,46127,7963,1,1111,4,0,485,3,154,1.600000,12,14,0,0.0
94597,46130,395,2,745,4,0,399,3,163,1.666667,12,20,6,0.0
94598,46130,395,2,745,4,0,399,3,163,1.666667,12,20,6,0.0


0.0    15034
1.0     1847
2.0     1194
3.0      848
4.0      618
Name: prediction, dtype: int64

In [44]:
# Make/addapt to submission file
submission_jan.drop(['prediction'], axis = 1, inplace = True) 
submission_jan = pd.merge(X_test, submission_jan, on=['userID', 'itemID'], how='right')

#Fill inn 0 for the (user, items) combinations we have not seen before (Not seen in january. Might have to small test set)
submission_jan['prediction'] = submission_jan['prediction'].fillna(0) 
submission_jan.drop_duplicates(subset=['userID', 'itemID'], keep='first', inplace = True)
submission_jan.drop(['order', 'dayofweek', 'month', 'day', 'brand', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5'], axis = 1, inplace = True)
display(submission_jan)

tmp5 = submission_jan.prediction.value_counts()
tmp5

,userID,itemID,avg_order_amount,prediction
0,0,20664,1.000000,0.0
3,0,28231,1.000000,0.0
7,13,2690,1.000000,4.0
11,15,1299,1.000000,0.0
17,15,20968,1.000000,0.0
...,...,...,...,...
24360,46118,20106,1.000000,2.0
24365,46124,19677,1.000000,1.0
24375,46125,12878,NaN,0.0
24376,46127,7963,1.600000,0.0


0.0    8892
1.0     341
2.0     287
3.0     239
4.0     181
Name: prediction, dtype: int64

In [45]:
# Make submission file for january
# submission_jan.to_csv('submission_jan_predict.csv', sep='|', index=False)

In [46]:
# Fin gold for januar.
orders = pd.read_csv('orders.csv', sep='|')

jan_ordered = orders[(orders['date'] >= '2021-01-01') & (orders['date'] <= '2021-12-28')]

# Right join the actual orders happening in january and "possible transactions" in january
jan_joined = pd.merge(jan_ordered, submission_jan, on=['userID', 'itemID'], how='right')

# Fill in correct predictions/goldstandard
jan_joined['prediction'] = 0

jan_joined['prediction'] = np.where((jan_joined['date'] >= '2021-01-01') & (jan_joined['date'] <= '2021-01-07'),1 ,jan_joined.prediction)
jan_joined['prediction'] = np.where((jan_joined['date'] >= '2021-01-08') & (jan_joined['date'] <= '2021-01-14'),2 ,jan_joined.prediction)
jan_joined['prediction'] = np.where((jan_joined['date'] >= '2021-01-15') & (jan_joined['date'] <= '2021-01-21'),3 ,jan_joined.prediction)
jan_joined['prediction'] = np.where((jan_joined['date'] >= '2021-01-22') & (jan_joined['date'] <= '2021-01-28'),4 ,jan_joined.prediction)

jan_joined.drop(['order', 'date'], axis = 1, inplace = True) 

# Take first week if person buyes same item multiple times in our test period
jan_joined.drop_duplicates(subset=['userID', 'itemID'], keep='first', inplace = True) 

y_gold = jan_joined
display(y_gold)

# Our data looks to be a bitt unbalanced with far more cases of not buying than buying in week 1, 2, 3 or 4.
tmp8 = y_gold.prediction.value_counts()
tmp8

,userID,itemID,avg_order_amount,prediction
0,0,20664,1.000000,0
1,0,28231,1.000000,4
2,13,2690,1.000000,0
3,15,1299,1.000000,2
4,15,20968,1.000000,4
...,...,...,...,...
10208,46118,20106,1.000000,0
10209,46124,19677,1.000000,4
10210,46125,12878,NaN,0
10211,46127,7963,1.600000,0


0    6419
1    1072
2     929
3     776
4     744
Name: prediction, dtype: int64

In [47]:
def count_points(pred, gold):
    df = pd.merge(pred, gold, on=['userID', 'itemID'], suffixes=('_pred', '_gold'))
    df['points'] = df.apply(_compute_points_for_row, axis=1)
    return df['points'].sum()

def _compute_points_for_row(row):
    y_pred, y_gold = row.prediction_pred, row.prediction_gold
    if y_pred == y_gold:
        # one point if "no order" (0) is predicted correctly; three points if order week is predicted correctly
        return 1 if y_pred == 0 else 3
    # one point if order is predicted correctly (but not the correct week), otherwise zero points
    return 1 if (y_pred > 0 and y_gold > 0) else 0

In [48]:
# 'pred' and 'gold' are DataFrames with columns ['userID', 'itemID', 'prediction']
pred, gold = submission_jan, y_gold  # TODO: load your prediction and goldstandard

points = count_points(pred, gold)
max_points = count_points(gold, gold)
score = points / max_points
print(score)

0.3574961724178542
